In [1]:
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ContentType
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types.message import ContentTypes
from aiogram.dispatcher import FSMContext
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from sqlalchemy import update
from aiogram.dispatcher.filters.state import State, StatesGroup

API_TOKEN = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY"


In [2]:
from food.paths import *
from food.search import *
import pandas as  pd
from food.psql import *
import pytz
timezones = pytz.all_timezones
import requests
from requests.structures import CaseInsensitiveDict
import urllib
from tzwhere import tzwhere

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

dishes_table = Dishes.__table__

In [5]:
from aiogram.utils.callback_data import CallbackData
import typing

In [6]:
def geocode(q):
    geocoding_key = '5d96ac126bcb462cb373297924ab2cb4'
    url = "https://api.geoapify.com/v1/geocode/search?"

    params = {"apiKey":geocoding_key, 
            "text":q}

    resp = requests.get(url + urllib.parse.urlencode(params)).json()
    return  pd.json_normalize(resp['features']).sort_values('properties.rank.importance',ascending = False)[['properties.lat','properties.lon']].iloc[0].to_list()
   

In [7]:
def get_tz(q):
    lat,lon = geocode(q)
    return tzwhere.tzwhere().tzNameAt(lat,lon)

In [8]:
async def async_get_tz(q):
    return get_tz(q)

In [9]:
async def async_search_image(url, env='prod'):
    return search_image(url,env)

In [10]:
async def async_geocode(q):
    return geocode(q)

In [11]:
async def  async_insert_on_conflict(*args, **qwargs):
    return insert_on_conflict(*args, **qwargs)

In [12]:
async def add_sender(message):
    sender = message['from'].to_python()
    sender = pd.DataFrame(sender,index=[0]).drop(columns =['is_bot'])
    await async_insert_on_conflict(sender,'users',unique_cols=['id'])

In [13]:
add_dish_cb   = CallbackData('add_dish', 'action')
measurment_cb = CallbackData('measurment', 'weight')
edit_dish_cb  = CallbackData('edit_dish', 'action')

ml_version = 0.2

In [14]:
def get_keyboard(t):
    markup = types.InlineKeyboardMarkup()
    if t == 'add_dish' :  
        markup.add(types.InlineKeyboardButton('add_dish', callback_data=add_dish_cb.new(action='yes')))
         
    elif t == 'measurment':

        markup = types.InlineKeyboardMarkup(row_width=8)
        btns_text = tuple([(str(p)) for p in range(10,510,10)])
        markup.add(*(types.InlineKeyboardButton(text, callback_data=measurment_cb.new(weight=text)) for text in btns_text))

    elif t == 'edit_dish':

        btns_text = ('remove','edit_weight','add again')
        markup.add(*(types.InlineKeyboardButton(text, callback_data=edit_dish_cb.new(action=text)) for text in btns_text))


    

    return markup 


In [15]:
#photo recieved
@dp.message_handler(content_types=ContentType.PHOTO)
async def process_photo(message: types.Message, state: FSMContext):
    await types.ChatActions.typing()

    await add_sender(message)

    photo  = message['photo'][-1]
    await photo.download(reference_images_path/photo['file_id'])
    image_url = await photo.get_url()
    dish = await async_search_image(url=image_url, env='prod')

    dish['photo_id']   = photo['file_id']
    dish['message_id'] = message['message_id']
    sender = message['from'].to_python()
    dish['user_id'] = sender['id']
    dish['ml_version'] = ml_version 
    dish['timestamp']=pd.Timestamp.utcnow()
    print(2)

    # async with state.proxy() as data: data['dish'] = dish.to_dict(orient = 'records')[0]

    dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)
    plot_numtients = dish[['energy','protein','carb','fat']].reset_index(drop=True)
    plot_numtients.index = ['']
    
    msg = plot_numtients.astype(int).to_string()
    async with state.proxy() as data: data['msg'] = msg
    await message.reply(msg, reply_markup=get_keyboard('add_dish'))

In [16]:
#add_dish pushed
@dp.message_handler(state='*')
@dp.callback_query_handler(add_dish_cb.filter(action=['yes']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str], state: FSMContext):
    await query.answer()
    async with state.proxy() as data:  msg = data['msg']
    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('measurment'),
    )

In [17]:
#measure provided
@dp.message_handler(state='*')
@dp.callback_query_handler(measurment_cb.filter(weight=[str(p) for p in range(10,510,10)]))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str], state: FSMContext):
    await query.answer()
    async with state.proxy() as data:  msg = data['msg']
    msg = f"{msg}\n consumed {callback_data['weight']} grams"
    async with state.proxy() as data:  data['msg'] = msg

    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('edit_dish')
    )
    

In [18]:
#remove pushed
@dp.message_handler(state='*')
@dp.callback_query_handler(edit_dish_cb.filter(action=['remove']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str], state: FSMContext):

    async with state.proxy() as data:  msg = data['msg']


    msg = ''.join(msg.split('\n')[:-1])
    msg = f"{msg}"
    async with state.proxy() as data:  data['msg'] = msg

    await query.answer()
    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('add_dish'))

In [19]:
# @dp.message_handler(state='*')
# @dp.callback_query_handler(edit_dish_cb.filter(action=['remove','edit_weight','add again']))
# async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str], state: FSMContext):

#     async with state.proxy() as data:  msg = data['msg']
#     msg = f"{msg}, eated {callback_data['weight']}"
#     async with state.proxy() as data:  data['msg'] = msg

#     await query.answer()
#     await bot.edit_message_text(
#         msg,
#         query.from_user.id,
#         query.message.message_id,
#         reply_markup=get_keyboard('edit_dish'))

In [20]:
types.InlineKeyboardMarkup(row_width=50)

<InlineKeyboardMarkup {"inline_keyboard":[]}>

In [21]:

if __name__ == '__main__':
    executor.start_polling(dp)

In [ ]:
m

NameError: name 'm' is not defined